# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Actividad Semana 5

### **Vectores Embebidos de OpenAI**

#### **Nombres y matrículas de los integrantes del equipo:**


*   Carolina Gómez Fernández - A01305369
*   Javier Alejandro Pérez Garza - A01284386
*   Jorge Andrés Santos Gordon - A01652587
*   Vanya Karime Betancourt Reyes - A01795070



In [ ]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:

import os

import re
import numpy as np
import pandas as pd

import nltk
import spacy

from nltk.corpus import stopwords, words
from nltk.stem import SnowballStemmer
from nltk import pos_tag
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
# Incluye las celdas necesarias para tu acceso a la API de OpenAI.
#!pip install openai
import openai
import pickle
import time
openai.api_key = #'key'

# **Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
DIR = '/content/drive/MyDrive/Colab Notebooks/MNA/TC5043 - Procesamiento de lenguaje natural/Semana 5'

os.chdir(DIR)


dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', delimiter='\s{2,}(?=\d)', names=['review','label'], header=None, engine ='python', encoding='utf-8')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')


df = pd.concat([dfa, dfi, dfy], ignore_index=True)

# *********** Aquí termina la sección de agregar código *************


Mounted at /content/drive


In [ ]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [ ]:
# Y veamos sus primeros registros:

df.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


# **Pregunta - 2:**

Realiza el proceso de limpieza. Aplica el preprocesamiento que consideres adecuado.











In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Consideremos la siguiente lista de palabras asociada a negaciones en inglés:

nltk_words = set(words.words())
nlp = spacy.load("en_core_web_sm")

negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

#Excluimos negwords de stopwords
mystopwords =  [x for x in stopwords.words('english') if x not in negwords]

X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s


In [ ]:
def clean_tok(doc):
  # Eliminamos todo lo que no sean letras
  words = re.sub(r'\b\w\b|[^a-zA-Z]',r' ' , doc)

  # Elimamos espacios en blanco duplicados
  words =  re.sub(r'\s+', ' ', words)

  # Aplicamos tecnica de lemmatizacion
  tokens = [wd.lemma_ for wd in nlp(words)]

  # Aplicamos stemming en palabras no válidas en inglés
  ss = SnowballStemmer("english")
  tokens = [token if token in nltk_words else ss.stem(token) for token in tokens]

  # Tokenizamos los strings y convertimos los token a minusculas
  tokens = [ token.strip().lower() for token in tokens]

  # Eliminamos Stopwords
  tokens = [token for token in tokens if token not in mystopwords]
  #
  # Eliminamos caracteres duplicados no validos
  tokens = [clean_dup_chars(token) for token in tokens]


  return tokens


def clean_dup_chars(word):

 if word not in nltk_words:
   if re.search(r'(.)\1', word):
     word = re.sub(r'([a-z])\1', r'\1', word[::-1], count=1)[::-1]
     return clean_dup_chars(word)
   else:
    return word
 else:
  return word



In [ ]:
# Aplicamos el proceso de limpieza/normalización adicionales:

Xclean = [clean_tok(x) for x in X ]

# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


['no', 'way', 'plug', 'us', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tie', 'charger', 'conversation', 'last', 'minute', 'major', 'problem']
['mic', 'great']


# **Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [ ]:

# ************* Inicia la sección de agregar código:*****************************

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)


# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


# **Pregunta - 4:**



Construye tu vocabulario a continuación


In [ ]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


midiccionario = Counter()

for k in range(len(x_train)):
  midiccionario.update(x_train[k])

min_freq = 3
min_word_len = 3

midiccionario = {token: freq for token, freq in midiccionario.items()
                 if freq >= min_freq and len(token) >= min_word_len}

# *********** Aquí termina la sección de agregar código *************

In [ ]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********


len(midiccionario)


# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:


907

c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

Porque si usamos los otros dos conjuntos, corremos el riesgo de tener filtrado de información (vería los inputs de test/train), lo cual afectaría las métricas de desempeño de los modelos al hacer prueba y validación, y dar una idea "inflada" de su verdadera precisión.

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [ ]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********



train_x = []
for ss in x_train:
  train_x.append([w for w in ss if w in midiccionario])

val_x = []
for ss in x_val:
  val_x.append([w for w in ss if w in midiccionario])

test_x = []
for ss in x_test:
  test_x.append([w for w in ss if w in midiccionario])


print('X,y Train:', len(train_x), len(y_train))
print('X,y Val:', len(val_x), len(y_val))
print('X,y Test', len(test_x), len(y_test))


# *********** Aquí termina la sección de agregar código *************


X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


In [ ]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

['star', 'don', 'much', 'well', 'people', 'like', 'waste']
['special', 'suck']
['pay', 'bill', 'not', 'tip', 'feel', 'server', 'terrible', 'job']
['call', 'cook', 'steak', 'don', 'understand']
['however', 'keypad', 'tinny', 'sometimes', 'wrong', 'button']


# **Pregunta - 5:**


#### **Incluye aquí un resumen de las características y diferencias que tiene al menos los tres modelos de OpenAI indicados: "text-embedding-3-small", "text-embedding-3-large" y "text-embedding-ada-002".**

### ++++++++ Inicia la sección de agregar texto: +++++++++++
Los siguientes tres modelos son opciones que OpenAI ofrece para hacer embeding de texto:
1. "text-embedding-ada-002": Este embedding tiene 1536 dimensiones, y su costo de procesamiento es bajo en comparación a otras opciones. Su precisión es menor a la de sus contrapartes, pero es útil para casos de uso básicos. Fue de los primeros modelos lanzados por OpenAI, en 2022.
2. "text-embedding-3-small": Lanzado en 2024, es la evolución del modelo ada-002. Su precisión es mayor, pero mantiene las mismas dimensiones (1536). El costo de procesamiento es similar al de ada-002, y permite ajustar la dimensión de la salida
3. "text-embedding-3-large": Es un complemento a su versión small, con 3072 dimensiones. Su precisión es la más alta de las 3 opciones, al igual que su costo. Es la mejor opción para tareas complejas, que requieren una comprensión profunda del texto y sus relaciones.

**Referencias:**

OpenAI. (2025). Vector Embeddings. Recuperado de https://platform.openai.com/docs/guides/embeddings#embedding-models

### ++++++++ Termina la sección de agregar texto: +++++++++++


# **Pregunta - 6:**


#### **Diccionario clave-valor de palabras del diccionario y vectores embebidos.**

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Generar lista ordenada
vocabulario = list(midiccionario.keys())

# Diccionario para almacenar los vectores de palabras
embedding_dict = {}

# Contador de tokens
total_tokens = 0
batch_size = 100

# Modelo a usar
embedding_model = "text-embedding-3-small"

# Llamadas a la API
for i in range(0, len(vocabulario), batch_size):
        batch = vocabulario[i:i+batch_size]
        response = openai.embeddings.create(
        input=batch,
        model=embedding_model)

        tokens_used = response.usage.total_tokens
        total_tokens = total_tokens + tokens_used

        for j, embedding_data in enumerate(response.data):
          palabra = batch[j]
          vector = embedding_data.embedding
          embedding_dict[palabra] = vector

        # Timer
        time.sleep(0.5)

# Guardar los embeddings y etiquetas
with open("embedding_dict.pkl", "wb") as f:
    pickle.dump(embedding_dict, f)

print("Total de palabras embebidas:", len(embedding_dict))
print("Tokens de OpenAI utilizados:", total_tokens)

# *********** Aquí termina la sección de agregar código *************

Total de palabras embebidas: 907
Tokens de OpenAI utilizados: 1237


# **Pregunta - 7:**



Generamos los vectores embebidos a partir de los conjuntos de entrenamiento, validación y prueba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


with open("embedding_dict.pkl", "rb") as f:
  embedding_dict = pickle.load(f)

dim = len(next(iter(embedding_dict.values())))

# Convertir train_x en vectores promediados
trainEmb = []
for review in train_x:
  review_vectors = []
  for word in review:
    if word in embedding_dict:
      review_vectors.append(embedding_dict[word])
  if len(review_vectors) > 0:
    emb = np.mean(review_vectors, axis=0)
  else:
    emb = np.zeros(dim)
  trainEmb.append(emb)

# Convertir val_x en vectores promediados
valEmb = []
for review in val_x:
  review_vectors = []
  for word in review:
    if word in embedding_dict:
      review_vectors.append(embedding_dict[word])
  if len(review_vectors) > 0:
    emb = np.mean(review_vectors, axis=0)
  else:
    emb = np.zeros(dim)
  valEmb.append(emb)

# Convertir test_x en vectores promediados
testEmb = []
for review in test_x:
  review_vectors = []
  for word in review:
    if word in embedding_dict:
      review_vectors.append(embedding_dict[word])
  if len(review_vectors) > 0:
    emb = np.mean(review_vectors, axis=0)
  else:
    emb = np.zeros(dim)
  testEmb.append(emb)

# Convertir listas a arreglos de numpy para entrenamiento posterior
trainEmb = np.array(trainEmb)
valEmb = np.array(valEmb)
testEmb = np.array(testEmb)

# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

Train-Emb: (2100, 1536)
Val-Emb: (450, 1536)
Test-Emb: (450, 1536)


# **Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


modeloLR = LogisticRegression(C = 25,
                              random_state = 1)

modeloLR.fit(trainEmb, y_train)


print('Logistic Regression Accuracy:')
print(f"Train: {modeloLR.score(trainEmb, y_train):.2%}")
print(f"Val:   {modeloLR.score(valEmb, y_val):.2%}")
print(f"Test:  {modeloLR.score(testEmb, y_test):.2%}")
print('\nClassification Report (Logistic Regression):')
print(classification_report(y_test, modeloLR.predict(testEmb)))

# *********** Aquí termina la sección de agregar código *************


Logistic Regression Accuracy:
Train: 86.57%
Val:   84.00%
Test:  80.89%

Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       216
           1       0.81      0.82      0.82       234

    accuracy                           0.81       450
   macro avg       0.81      0.81      0.81       450
weighted avg       0.81      0.81      0.81       450



In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


modeloRF = RandomForestClassifier(n_estimators = 200,
                                  min_samples_split = 10,
                                  min_samples_leaf = 15,
                                  max_leaf_nodes = 10,
                                  max_features='sqrt',
                                  max_depth=5,
                                  n_jobs = -1,
                                  random_state = 42)

modeloRF.fit(trainEmb,y_train)


print('Random Forest Accuracy:')
print(f"Train: {modeloRF.score(trainEmb, y_train):.2%}")
print(f"Val:   {modeloRF.score(valEmb, y_val):.2%}")
print(f"Test:  {modeloRF.score(testEmb, y_test):.2%}")
print('\nClassification Report (Random Forest):')
print(classification_report(y_test, modeloRF.predict(testEmb)))


# *********** Aquí termina la sección de agregar código *************

Random Forest Accuracy:
Train: 85.24%
Val:   80.89%
Test:  75.78%

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.73      0.78      0.76       216
           1       0.79      0.74      0.76       234

    accuracy                           0.76       450
   macro avg       0.76      0.76      0.76       450
weighted avg       0.76      0.76      0.76       450



# **Pregunta - 9:**



Reporte del mejor modelo con el conjunto de Prueba (Test).


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

mejor_modelo = modeloLR

print('Test-accuracy con el mejor modelo %.2f%%' % (100*mejor_modelo.score(testEmb, y_test)))

pred = mejor_modelo.predict(testEmb)
print('\nMatriz de confusión con el mejor modelo:')
print(confusion_matrix(y_test, pred, labels=[0,1]))

print('\nMatriz de confusión con el mejor modelo en proporciones:')
print(confusion_matrix(y_test, pred, labels=[0,1]) / (pred.shape[0] ))

report = classification_report(y_test, pred)
print(report)

# *********** Aquí termina la sección de agregar código *************

Test-accuracy con el mejor modelo 80.89%

Matriz de confusión con el mejor modelo:
[[172  44]
 [ 42 192]]

Matriz de confusión con el mejor modelo en proporciones:
[[0.38222222 0.09777778]
 [0.09333333 0.42666667]]
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       216
           1       0.81      0.82      0.82       234

    accuracy                           0.81       450
   macro avg       0.81      0.81      0.81       450
weighted avg       0.81      0.81      0.81       450



# **Pregunta - 10:**

In [ ]:
# Incluye todas las líneas de código y celdas que consideres adecuadas para este ejercicio.

#a
# Generar lista ordenada
Xfull_text = list(X)

In [ ]:
#Lista para almacenar los vectores de Xfull_text
full_embeddings = []

# Contador de tokens
total_tokens = 0
batch_size = 100

# Modelo a usar
embedding_model = "text-embedding-3-small"

# Llamadas a la API
for i in range(0, len(Xfull_text), batch_size):
    batch = Xfull_text[i:i+batch_size]
    response = openai.embeddings.create(
        input=batch,
        model=embedding_model)

    total_tokens = total_tokens + response.usage.total_tokens
    for embedding_data in response.data:
        full_embeddings.append(embedding_data.embedding)

    # Timer
    time.sleep(0.5)

# Guardar los embeddings y etiquetas
with open("full_embedding_dict.pkl", "wb") as f:
    pickle.dump({"embeddings": full_embeddings, "labels": list(Y)}, f)


print("Tokens de OpenAI utilizados:", total_tokens)


Tokens de OpenAI utilizados: 43804


In [ ]:
#b
with open("full_embedding_dict.pkl", "rb") as f:
    data = pickle.load(f)

X_all = np.array(data["embeddings"])
Y_all = np.array(data["labels"])

# Partición
x_train, x_val_test, y_train, y_val_test = train_test_split(X_all, Y_all, train_size=0.70, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.5, random_state=17)

In [ ]:
#c
#Logistic Regression
modeloLR = LogisticRegression(C=25, random_state=1)
modeloLR.fit(x_train, y_train)

print('Logistic Regression Accuracy:')
print(f"Train: {modeloLR.score(x_train, y_train):.2%}")
print(f"Val:   {modeloLR.score(x_val, y_val):.2%}")
print(f"Test:  {modeloLR.score(x_test, y_test):.2%}")
print('\nClassification Report (Logistic Regression):')
print(classification_report(y_test, modeloLR.predict(x_test)))

Logistic Regression Accuracy:
Train: 99.62%
Val:   97.56%
Test:  97.78%

Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       216
           1       0.98      0.97      0.98       234

    accuracy                           0.98       450
   macro avg       0.98      0.98      0.98       450
weighted avg       0.98      0.98      0.98       450



In [ ]:
# Random Forest
modeloRF = RandomForestClassifier(
    n_estimators=200,
    min_samples_split=10,
    min_samples_leaf=15,
    max_leaf_nodes=10,
    max_features='sqrt',
    max_depth=5,
    n_jobs=-1,
    random_state=42
)
modeloRF.fit(x_train, y_train)

print('Random Forest Accuracy:')
print(f"Train: {modeloRF.score(x_train, y_train):.2%}")
print(f"Val:   {modeloRF.score(x_val, y_val):.2%}")
print(f"Test:  {modeloRF.score(x_test, y_test):.2%}")
print('\nClassification Report (Random Forest):')
print(classification_report(y_test, modeloRF.predict(x_test)))

Random Forest Accuracy:
Train: 98.19%
Val:   95.78%
Test:  96.89%

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       216
           1       0.98      0.96      0.97       234

    accuracy                           0.97       450
   macro avg       0.97      0.97      0.97       450
weighted avg       0.97      0.97      0.97       450



# **Pregunta - 11:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

Se trabajaron 3 archivos CVS llamados amazon5txt, yelp5.txt e imdb5.txt de Canvas. Se generó un solo DataFrame de Pandas con ellos. En particular. El archivo imdb no requirió transformarse para obtener sus 1000 registros. Con los datos obtenidos se realizó un proceso de limpieza.

El dataset fue dividido en conjuntos de entrenamiento 70%, validación 15% y prueba 15%, siguiendo las prácticas adecuadas en el entrenamiento de modelos supervisados.

Con los datos preparados se construyó el vocabulario, en la pregunta 4 se concluyé que lo mejor es utilizar el conjunto de entrenamiento para evitar sesgos por fuga de datos.

Después, se analizaron diferentes modelos de vectores embebidos de OpenAI, llegando a la conclusión que el modelo embedding-3-small era el que se iba a utilizar en el resto de las preguntas de este ejercicio.

Posteriormente, se entrenaron dos modelos de clasificación: Regresión Logística y Bosque Aleatorio (Random Forest) y se hizo una comparación entre la limpieza de datos hecha por nosotros y la limpieza hecha por el modelo de embedding de OpenAI. Al utilizar el embedding de Open AI se puede observar que ambos modelos obtuvieron resultados de precisión y recall elevados, que rondan al 98%, lo que sugiere que los embeddings generados capturan muy bien la información necesaria para clasificar los textos.

Se utilizó el reporte de clasificación (precision, recall, f1-score, support) para evaluar el desempeño de los modelos, mostrando métricas balanceadas para ambas clases (0 y 1), lo que indica un modelo robusto y bien entrenado.

Como conclusión general el uso de embeddings generados por modelos de OpenAI permite aprovechar representaciones semánticas ricas del lenguaje sin necesidad de entrenar modelos complejos desde cero, además la actividad permite regorzar las buenas prácticas en ciencia de datos, como la separación de datos, el uso de validación cruzada implícita, y la comparación objetiva entre modelos.
### ++++++++ Termina la sección de agregar texto: +++++++++++

# **Fin de la Actividad de Vectores Embebidos - OpenAI**